<a href="https://colab.research.google.com/github/Emc2Ikeda/recipe-recommender/blob/main/ai_recipe_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data set
URL: https://www.kaggle.com/datasets/thedevastator/better-recipes-for-a-better-life \\

Relevant columns: Ingredients \\

For dietary restrictions: determine whether the recipe meets dietary restrictions based on the ingredients list. For example, you know that refried beans recipe is vegan if the recipe does not use lard. \\

  Question: what learning algorithm should be used to have AI recognize that
the recipe meets dietary restrictions?

## Load and inspect the dataset

In [ ]:
# Imports
import numpy as np
import pandas as pd

In [ ]:
#Load CSV into DataFrame
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recipes.csv')
df.head()

,Unnamed: 0,recipe_name,prep_time,cook_time,total_time,servings,yield,ingredients,directions,rating,url,cuisine_path,nutrition,timing,img_src
0,0,Apple-Cranberry Crostada,NaN,NaN,NaN,8,6 to 8 - servings,"3 tablespoons butter, 2 pounds Granny Smith ap...",Heat butter in a large skillet over medium-hig...,4.4,https://www.allrecipes.com/recipe/76931/apple-...,/Desserts/Fruit Desserts/Apple Dessert Recipes/,"Total Fat 18g 23%, Saturated Fat 7g 34%, Chole...","Servings: 8, Yield: 6 to 8 - servings",https://www.allrecipes.com/thmb/Tf1wH73bfH6Oql...
1,1,Apple Pie by Grandma Ople,30 mins,1 hrs,1 hrs 30 mins,8,1 9-inch pie,"8 small Granny Smith apples, or as needed, ½ c...","Peel and core apples, then thinly slice. Set a...",4.8,https://www.allrecipes.com/recipe/12682/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 19g 24%, Saturated Fat 9g 46%, Chole...","Prep Time: 30 mins, Cook Time: 1 hrs, Total Ti...",https://www.allrecipes.com/thmb/1I95oiTGz6aEpu...
2,2,Sarah's Homemade Applesauce,10 mins,15 mins,25 mins,4,NaN,"4 apples - peeled, cored and chopped, ¾ cup w...","Combine apples, water, sugar, and cinnamon in ...",4.8,https://www.allrecipes.com/recipe/51301/sarahs...,/Side Dish/Applesauce Recipes/,"Total Fat 0g 0%, Sodium 3mg 0%, Total Carbohyd...","Prep Time: 10 mins, Cook Time: 15 mins, Total ...",https://www.allrecipes.com/thmb/VY5d0tZHB8xz6y...
3,3,Apple Crisp,30 mins,45 mins,1 hrs 15 mins,12,1 9x13-inch pan,"10 cups all-purpose apples, peeled, cored and ...",Preheat the oven to 350 degrees F (175 degrees...,4.7,https://www.allrecipes.com/recipe/12409/apple-...,/Desserts/Crisps and Crumbles Recipes/Apple Cr...,"Total Fat 8g 11%, Saturated Fat 5g 25%, Choles...","Prep Time: 30 mins, Cook Time: 45 mins, Total ...",https://www.allrecipes.com/thmb/uAzhPOh86PfR-N...
4,4,Apple Pie Filling,20 mins,20 mins,2 hrs 40 mins,40,5 9-inch pies,"18 cups thinly sliced apples, 3 tablespoons le...",Toss apples with lemon juice in a large bowl a...,4.7,https://www.allrecipes.com/recipe/12681/apple-...,/Desserts/Pies/Apple Pie Recipes/,"Total Fat 0g 0%, Sodium 61mg 3%, Total Carbohy...","Prep Time: 20 mins, Cook Time: 20 mins, Additi...",https://www.allrecipes.com/thmb/c0bbYaS1V_mTt_...


##Check the dataframe for null values and duplicates in recipe_name and ingredients column

In [ ]:
# check the DataFrame for columns, null values and duplicates.
#recipe_name had some duplicate values. ingredients column did not have null or duplicate values
# df.info()

# null_ingredients = int(df['ingredients'].isnull().sum())
# duplicate_ingredients = int(df.duplicated().sum())

# null_names = int(df['recipe_name'].isnull().sum())
# duplicate_names = int(df['recipe_name'].duplicated().sum())
# print(f"recipe_name - Null values: {null_names}, Duplicates: {duplicate_names} ")
# print(f"ingredients - Null values: {null_ingredients}, Duplicates: {duplicate_ingredients} ")

In [ ]:
# Find and delete recipes with duplicated names.

# Find recipe names that appear more than once
name_counts = df['recipe_name'].value_counts()
duplicates = name_counts[name_counts > 1]

# Display them
# print(f"Duplicate recipe names:\n{duplicates}")

# # Filter rows with duplicate recipe names
# df_duplicates = df[df['recipe_name'].isin(duplicates.index)]

# # Sort for easier viewing
# df_duplicates = df_duplicates.sort_values('recipe_name')

# # View a sample
# df_duplicates[['recipe_name', 'ingredients']].tail(10)

In [ ]:
# # check if duplicate recipes also have the same ingredient list (true duplicates)

# # Group by recipe name and count unique ingredients per group
# duplicates_with_diff = df.groupby('recipe_name')['ingredients'].nunique()

# # Filter to show recipe names that have more than 1 unique ingredients entry
# conflicting_duplicates = duplicates_with_diff[duplicates_with_diff > 1]

# # Display them
# print(f"Number of recipe names with different ingredients: {len(conflicting_duplicates)}")
# print(conflicting_duplicates.head())

In [ ]:
# Since all the recipes with duplicate names also have duplicate ingredient list, drop recipes with duplicate names.
df = df.drop_duplicates(subset='recipe_name').reset_index(drop=True)
print(f"Remaining duplicate names: {df['recipe_name'].duplicated().sum()}")

Remaining duplicate names: 0


## Format ingredients column to a list

In [ ]:
#Preprocess ingredients column in DataFrame to create a list of ingredients for the algorithm to use
import re

cleaned_ingredients = []

for row in df['ingredients']:
  ingredients = row.lower()
  ingredients = re.sub(r'[^\w\s,]','',ingredients).split(',')
  ingredients = [item.strip() for item in ingredients if item.strip()]
  cleaned_ingredients.append(ingredients)

df['ingredients_clean'] = cleaned_ingredients
print(cleaned_ingredients[0])

['3 tablespoons butter', '2 pounds granny smith apples or other firm', 'crisp apples', 'peeled', 'quartered', 'cored and sliced 14inch thick', '1 pound macintosh apples or other softtextured apples that fall apart when cooked', 'peeled', 'quartered', 'cored', 'and sliced 14inch thick', '½ cup sugar', '½ cup dried cranberries', '1 sheet frozen puff pastry', 'thawed but still cold follow package directions', '1  egg white', 'lightly beaten', '1 tablespoon sugar', '1 cup optional ice cream or lightly sweetened whipped cream']


In [ ]:
# show example of ingredients column. They are a string of comma-separated items, lower case except for proper capitalization (e.g. Granny Smith apples)
df['ingredients'].head(2)

,ingredients
0,"3 tablespoons butter, 2 pounds Granny Smith ap..."
1,"8 small Granny Smith apples, or as needed, ½ c..."


# Implement Backend Functions

In [ ]:
# === Constants: Dietary Rules and Ingredient Traits ===
# Ingredient tags of common allergens and dietary restriction foods
INGREDIENT_TRAITS = {
    "dairy": ['milk', 'cheese', 'butter', 'cream', 'yogurt', 'ghee'],
    "meat": ['chicken', 'beef', 'pork', 'lamb', 'bacon', 'chicken stock', 'beef stock'],
    "seafood": ['fish', 'shrimp', 'crab', 'lobster', 'anchovy', 'squid', 'clam', 'surimi'],
    "eggs": ['egg', 'egg whites', 'egg yolk'],
    "gluten": ['wheat', 'barley', 'rye', 'malt'],
    "nuts": ['almonds', 'peanuts', 'walnuts', 'cashews'],
    "animal_products": ['gelatin', 'honey', 'lard'],  # for vegan
    "soy": ['soy', 'tempeh', 'tofu', 'soy milk', 'soy sauce', 'miso', 'edamame']
}

# Dietary restrictions from ingredient tags. EX: vegan diet would exclude meat, seafood, dairy, and animal products
DIETARY_RESTRICTIONS = {
    "vegan": ['dairy', 'meat', 'seafood', 'eggs', 'animal_products'],
    "vegetarian": ['meat', 'seafood'],
    "gluten_free": ['gluten'],
    "nut_free": ['nuts'],
    "dairy_free": ['dairy']
}

In [ ]:
"""
Accepts a list of user-provided restrictions (e.g. 'vegan', 'dairy', 'soy')
and returns a full list of excluded ingredients.
"""
def get_excluded_ingredients(selected_diets):
  excluded = set()
  for item in selected_diets:
    item = item.lower().strip()

    if item in DIETARY_RESTRICTIONS:
      traits = DIETARY_RESTRICTIONS.get(item, [])
      for trait in traits:
          excluded.update(INGREDIENT_TRAITS.get(trait, []))

    elif item in INGREDIENT_TRAITS:
      excluded.update(INGREDIENT_TRAITS[item])

    else:
      excluded.add(item)
  return excluded

In [ ]:
# Filter out recipes in the dataset containing ingredients excluded by the user
def filter_recipes(recipes, excluded_ingredients):
    filtered = []
    for recipe in recipes:
        ingredients = [i.lower() for i in recipe['ingredients']]
        if not any(ingredient in ingredients for ingredient in excluded_ingredients):
            filtered.append(recipe)
    return filtered

In [ ]:
# TODO: Incorporate this to the rest of the source code. Reorganize cells in this project first

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

"""
    Recommend top 5 recipes based on user-provided ingredients and dietary restrictions.
    This function assumes that recipes are filtered to remove recipes that contain excluded ingredients.
    Uses bag-of-words model with cosine similarity to match the user's desired ingredients against the ingredient list of recipes.

    Args:
       1. recipes (list of dict): Recipe dataset filtered by excluded ingredients
       2. included_ingredients (list of str): Ingredients the user wants to include
    Returns:
       List of dict: Top 5 recipe dictionaries ranked by similarity to the included ingredients.
"""
def recommend_recipe(recipes, included_ingredients):
  texts = [' '.join(r['ingredients']) for r in recipes]

  vectorizer = CountVectorizer()
  recipe_vectors = vectorizer.fit_transform(texts)

  user_query = ' '.join(included_ingredients)
  user_vector = vectorizer.transform([user_query])

  similarities = cosine_similarity(user_vector, recipe_vectors).flatten()
  ranked = sorted(zip(recipes, similarities), key=lambda x:x[1], reverse=True)
  return ranked[:5]


# Implement UI

For future iterations: consider using AI to guess user's restricted ingredients.

In [ ]:
# Prompt the user to enter several ingredients and dietary restrictions. Conver this to individual ingredients to exclude in the recipe.
ingredients_input = input("Enter the ingredients (comma separated): ")
included_ingredients = [token.strip().lower() for token in ingredients_input.split(',') if token.strip()]

excluded_input = input("Enter ingredients to exclude. You can also enter dietary restrictions (comma separated): ")
excluded_tokens = [token.strip().lower() for token in excluded_input.split(',') if token.strip()]

selected_diets = [] # ingredients excluded under known dietary restrictions

for token in excluded_tokens:
  selected_diets.append(token)

excluded_ingredients = get_excluded_ingredients(selected_diets)

print(f"\nIngredients to include: {included_ingredients}")
print(f"Ingredients to exclude (including dietary restrictions): {sorted(excluded_ingredients)}")



Enter the ingredients (comma separated): apple
Enter ingredients to exclude. You can also enter dietary restrictions (comma separated): dairy

Ingredients to include: ['apple']
Ingredients to exclude (including dietary restrictions): ['butter', 'cheese', 'cream', 'ghee', 'milk', 'yogurt']


In [ ]:

filter_recipes(df['ingredients_clean'], excluded_ingredients)
# top_5_recipes = recommend_recipe(filter_recipes(df['ingredients_clean'], excluded_ingredients), included_ingredients)
# print(top_5_recipes)

pandas.core.series.Series